In [1]:
# импортируем бэкенд Agg из matplotlib для сохранения графиков на диск
import matplotlib
matplotlib.use("Agg")
# подключаем необходимые пакеты
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import os


data = []
labels = []
train_dir = os.path.join(os.getcwd(), 'train')

# цикл по изображениям из папки train - вытаскиваем картинки и кладем в data, их класс (имя симпсона) в labels
for dir in os.listdir(train_dir):
    cur_dir = os.path.join(train_dir, dir)
    print(f'[INFO] loading {dir}...')
    for image_path in os.listdir(cur_dir):
        # загружаем изображение, меняем размер на 32x32 пикселей (без учёта
        # соотношения сторон), сглаживаем его в 32x32x3=3072 пикселей и
        # добавляем в список
        try:
            global_image_path = os.path.join(cur_dir, image_path)
            image = cv2.imread(global_image_path)
            image = cv2.resize(image, (32, 32)).flatten()
            data.append(image)
        # извлекаем метку класса из пути к изображению и обновляем
        # список меток
        except Exception as e:
            print(str(e))
        labels.append(dir)

# масштабируем интенсивности пикселей в диапазон [0, 1]
data = np.array(data, dtype="float") / float(255)
labels = np.array(labels)

# разбиваем данные на обучающую и тестовую выборки
seed = 125 # random.randint(1, 100)
print("seed =", seed)
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, random_state=seed)

# конвертируем метки из целых чисел в векторы
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# определим архитектуру нейронной сети с помощью Keras
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(5, 5), padding="same", activation="relu", input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation="sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(64, activation="sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(len(lb.classes_), activation="softmax"))

# компилируем модель, используя Adam как оптимизатор и категориальную кросс-энтропию в качестве функции потерь
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# архитектура нейронки
model.summary()

# обучаем нейросеть
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=25, batch_size=32)

# оцениваем нейросеть
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

# строим графики потерь и точности
plt.style.use("ggplot")
N = np.arange(0, 25)
plt.figure()
plt.plot(N, H.history["loss"], label="training loss")
plt.plot(N, H.history["val_loss"], label="validation loss")
plt.plot(N, H.history["accuracy"], label="training accuracy")
plt.plot(N, H.history["val_accuracy"], label="validation accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('plot_con.png')
plt.show()

# начинаем предсказывать принадлежность к классу
test_labels = os.listdir('test')
test_labels.sort()

# цикл по изображениям test/
for test_file in test_labels:
    path_for_save = 'result/' + test_file
    test_file = 'test/' + test_file
    image = cv2.imread(test_file)
    image_copy = image.copy()
    image = cv2.resize(image, (32, 32))
    # масштабируем значения пикселей к диапазону [0, 1]
    image = image.astype("float") / 255.
    image = image.reshape((1, *image.shape))
    # массив предсказания класса картинки
    predictions = model.predict(image)
    # находим индекс класса с наибольшей вероятностью
    i = predictions.argmax(axis=1)[0]
    label = lb.classes_[i]
    # выводим название класса и вероятность принадлежности на картинку
    text = f'{label}: {round(predictions[0][i] * 100, 2)}%'
    cv2.putText(image_copy, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    plt.imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
    plt.show()
    cv2.imwrite(path_for_save, image_copy)


[INFO] loading krusty...
OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-vi271kac\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-vi271kac\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-vi271kac\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-vi271kac\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-vi271kac\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5

ValueError: Found input variables with inconsistent numbers of samples: [0, 373]